In [20]:
# import resnet 18
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import io
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.mixture import GaussianMixture
import numpy as np
import os
import csv
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from time import perf_counter


In [22]:
def read_image(image_path):
    img = io.read_image(image_path)

    weights = ResNet18_Weights.DEFAULT
    transform = weights.transforms()
    return transform(img)

def train(model, data, optimizer):
    pass
    train_loss, test_accuracy = 0, 0
    return train_loss, test_accuracy

def test(model, data):
    pass
    test_loss, test_accuracy = 0, 0
    return test_loss, test_accuracy
    


def train_loop(images, num_epochs=15, batch_size=32, learning_rate=0.001, weight_decay=0.0001):
    # load the images
    images = ...
    
    # train test split
    train_set = ...
    test_set = ...
    
    resnet = resnet18(weights=ResNet18_Weights.DEFAULT)
    for param in resnet.parameters():
      param.requires_grad = False
    # re-initialize final fully-connected layer
    num_features = resnet.fc.in_features
    resnet.fc = nn.Linear(num_features, 6)
    
    epoch = 0
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []
    
    while epoch < num_epochs:
        optimizer = optim.Adam(resnet.fc.parameters(), lr=learning_rate, weight_decay=weight_decay)
        
        train_loss, train_accuracy = train(...)
        test_loss, test_accuracy = test(...)
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        
        torch.save(resnet, 'model')
        
    
    # TODO: plot the training process
        
        
        
